In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
import numpy as np
import time
import pandas as pd
import tensorflow as tf

In [ ]:
train = pd.read_csv("../input/ventilator-pressure-prediction/train.csv")
test = pd.read_csv("../input/ventilator-pressure-prediction/test.csv")
submission = pd.read_csv("../input/ventilator-pressure-prediction/sample_submission.csv")

train.head()

In [ ]:
df_train = train.iloc[:,2:]
df_test = test.iloc[:,2:]

df_test.head(5)

In [ ]:

def prep_data_TRAIN(data, batch_sz):
    
    dataset_feat = tf.data.Dataset.from_tensor_slices(data.iloc[:,:-1])
    dataset_label = tf.data.Dataset.from_tensor_slices(data.iloc[:,-1])

    window_feat = dataset_feat.window(80,  drop_remainder=True)
    window_label = dataset_label.window(80,  drop_remainder=True)

    feat = window_feat.flat_map(lambda window: window.batch(80))
    label = window_label.flat_map(lambda window: window.batch(80))

    dataset = tf.data.Dataset.zip((feat, label)) 

    dataset = dataset.batch(batch_sz).prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [ ]:
def prep_data_TEST(data, batch_sz):
    
    dataset_feat = tf.data.Dataset.from_tensor_slices(data)

    window_feat = dataset_feat.window(80,  drop_remainder=True)

    feat = window_feat.flat_map(lambda window: window.batch(80))

    dataset = tf.data.Dataset.zip(feat) 
    
    dataset = dataset.batch(batch_sz).prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

In [ ]:
batch_sz_tr = 64
batch_sz_ts = 1

In [ ]:
train_dl = prep_data_TRAIN(df_train, batch_sz_tr)
test_dl = prep_data_TEST(df_test, batch_sz_ts)

In [ ]:
print('TRAIN')
print('batch features shape')
print(list(train_dl.take(1))[0][0].shape)
print('batch label shape')
print(list(train_dl.take(1))[0][1].shape)

In [ ]:
print('TEST')
print('batch features shape')
print(list(test_dl.take(1))[0].shape)
